In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import PIL 
import glob
from PIL import Image
import os ,sys
import batch

In [7]:
%matplotlib inline

# Make Train Batch

In [19]:
train_1=glob.glob("/ssd/cervical/train/Type_1/*.jpg")
train_2=glob.glob("/ssd/cervical/train/Type_2/*.jpg")
train_3=glob.glob("/ssd/cervical/train/Type_3/*.jpg")

In [20]:
additional_1=glob.glob("/ssd/cervical/train/additional_1/*.jpg")
additional_2=glob.glob("/ssd/cervical/train/additional_2/*.jpg")
additional_3=glob.glob("/ssd/cervical/train/additional_3/*.jpg")

In [39]:
def makeBatch(*args):
    imgs=[]
    for thing in args:
        paths=list(thing) 
        for path in paths:
            try:
                img=Image.open(path)
                img=resize_image(img)
                imgs.append(img)
            except IOError as ioe:
                f=open('Error_log.txt' , 'a')
                f.write(str(ioe))
                f.write(path)
                print path
                continue
            
    return imgs

In [22]:
def resize_image(img , size =(224,224)):
    img=img.resize(size , Image.ANTIALIAS)
    return img

In [37]:
def Images2numpy(images , img_size=(224,224) , color_ch =3 ):
    n=len(images)
    imgs_np=np.zeros([n,img_size[0] , img_size[1] , color_ch ])
    for i,image in enumerate(images):
        imgs_np[i] = image
    return imgs_np
        

In [40]:
train_1_imgs=makeBatch(train_1 , additional_1)
#train_2_imgs=makeBatch(train_2 , additional_2)
#train_3_imgs=makeBatch(train_3 , additional_3)

/ssd/cervical/train/Type_1/1339.jpg


In [41]:
train_1_np = Images2numpy(train_1_imgs)

In [42]:
np.shape(train_1_np)

(249, 224, 224, 3)

In [8]:
batch_=batch.batch('/ssd/cervical/')

NameError: global name 'crawl_folder' is not defined